In [1]:
import time
import sys
sys.path.append("/home/boxinw-src/megatron-lm/megatron")
sys.path.append("/home/boxinw-src/megatron-lm/")

import h5py
import numpy as np
from tqdm import tqdm

import os, psutil

import argparse


In [2]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss / 1024 / 1024 / 1024)  # in bytes

0.07331466674804688


In [3]:
import faiss                   # make faiss available
ngpus = faiss.get_num_gpus()

AttributeError: module 'faiss' has no attribute 'get_num_gpus'

In [4]:
print("number of GPUs:", ngpus)


number of GPUs: 16


In [5]:
data_path = 'pretraining_index_OPQ64_128_IVF4194304_HNSW32_PQ64_16_16.bin'

start = time.time()
index = faiss.read_index(data_path)
end = time.time()

print("time cost:", end - start)

time cost: 572.5671734809875


In [6]:
from faiss import ParameterSpace

ParameterSpace().set_index_parameter(index, "efSearch", 256)
ParameterSpace().set_index_parameter(index, "nprobe", 65536)  # potentially larger nprobe?

In [7]:
input_path = '/gpfs/fs1/projects/gpu_adlr/datasets/boxinw/pretrained_data/Wikipedia-shuf/Wikipedia_en_ftfy_id_shuf_text_document.tokens.0000.feat.hdf5'

start = time.time()
f = h5py.File(input_path, "r")
features = np.copy(f['feat'])
f.close()
end = time.time()

print("time cost:", end - start)

time cost: 14.404329538345337


In [8]:
print(features.shape)


(4074624, 1024)


In [9]:
feature = features[:40746]

In [10]:
start = time.time()
k = 2                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[1335466853 1344065098]
 [1335466854 1338353227]
 [1335466855 1335466856]
 ...
 [ 897467922 2576169652]
 [1349281123 1324398266]
 [1686141679 1358354506]]
[[ 1.3314159  4.8101816]
 [ 2.3709533  4.767527 ]
 [ 2.4757743  4.67912  ]
 ...
 [ 4.972886   5.0007386]
 [ 4.0512795  4.0792313]
 [10.425038  10.818621 ]]
time cost: 206.52368783950806


In [12]:
start = time.time()
k = 10                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[1335466853 1344065098 1381864259 ... 1322928044  404059385 1318052923]
 [1335466854 1338353227 1361070531 ... 1357536321 1361323718 1362546185]
 [1335466855 1335466856 1335466857 ... 1346595726 1327594547 1325934529]
 ...
 [ 897467922 2576169652 3109787930 ... 1545270208  910139135 1922323886]
 [1349281123 1324398266 1334571839 ... 1349019295 1378832907 1339300472]
 [1686141679 1358354506 1356905531 ... 1329841808 1356247521 1358206592]]
[[ 1.3314159  4.8101816  4.849719  ...  5.0900664  5.114468   5.34592  ]
 [ 2.3709533  4.767527   4.970966  ...  5.314371   5.3506274  5.3780127]
 [ 2.4757743  4.67912    4.856323  ...  5.319588   5.351542   5.3570805]
 ...
 [ 4.972886   5.0007386  5.1257625 ...  5.4280434  5.572723   5.591442 ]
 [ 4.0512795  4.0792313  4.126018  ...  4.3210793  4.349822   4.44863  ]
 [10.425038  10.818621  11.25877   ... 11.830033  11.839164  11.9384775]]
time cost: 178.35554385185242


In [13]:
start = time.time()
k = 20                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[1335466853 1344065098 1381864259 ... 1327594546 1338822945 1380858621]
 [1335466854 1338353227 1361070531 ... 1380443164 1328682204 1376009224]
 [1335466855 1335466856 1335466857 ... 1380858621 1353877500 1362261166]
 ...
 [ 897467922 2576169652 3109787930 ... 2510454581 1369129709 3944630352]
 [1349281123 1324398266 1334571839 ... 1341246863 1356064365 1339427141]
 [1686141679 1358354506 1356905531 ... 1320227191 1373220521 1339146002]]
[[ 1.3314159  4.8101816  4.849719  ...  5.567738   5.616089   5.6630816]
 [ 2.3709533  4.767527   4.970966  ...  5.8402934  5.867517   5.880939 ]
 [ 2.4757743  4.67912    4.856323  ...  5.638487   5.6632295  5.739858 ]
 ...
 [ 4.972886   5.0007386  5.1257625 ...  5.8862286  5.9124117  5.918078 ]
 [ 4.0512795  4.0792313  4.126018  ...  4.598728   4.619939   4.628723 ]
 [10.425038  10.818621  11.25877   ... 12.182884  12.19228   12.193619 ]]
time cost: 174.63433742523193


In [11]:
start = time.time()
k = 200                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[1335466853 1344065098 1381864259 ... 1375383932 1325599290 3528679615]
 [1335466854 1338353227 1361070531 ... 1354708499 1362104966 1323123993]
 [1335466855 1335466856 1335466857 ... 1359595874  838046148 3516228237]
 ...
 [ 897467922 2576169652 3109787930 ... 1564815426 3367733197 3622870990]
 [1349281123 1324398266 1334571839 ... 1368913217 1334735734 1363600853]
 [1686141679 1358354506 1356905531 ... 1321201503 4212922217 1363137175]]
[[ 1.3314159  4.8101816  4.849719  ...  7.0967736  7.100786   7.101558 ]
 [ 2.3709533  4.767527   4.970966  ...  7.3665123  7.3671412  7.3713684]
 [ 2.4757743  4.67912    4.856323  ...  7.0940685  7.0977473  7.097836 ]
 ...
 [ 4.972886   5.0007386  5.1257625 ...  6.7238293  6.724666   6.725226 ]
 [ 4.0512795  4.0792313  4.126018  ...  5.7428036  5.744847   5.749653 ]
 [10.425038  10.818621  11.25877   ... 13.476592  13.488592  13.514265 ]]
time cost: 180.1122260093689


In [12]:
start = time.time()
k = 2000                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[1335466853 1344065098 1381864259 ... 1381724146  937015705 1324020770]
 [1335466854 1338353227 1361070531 ... 1338602645 1340925617 1365704499]
 [1335466855 1335466856 1335466857 ... 1339076546 1324020753 1344029892]
 ...
 [ 897467922 2576169652 3109787930 ... 3739607710 1351920439 3820561013]
 [1349281123 1324398266 1334571839 ... 1354789186 1371432701 1378489013]
 [1686141679 1358354506 1356905531 ... 1359142156 1362900137 3198554715]]
[[ 1.3314159  4.8101816  4.849719  ...  9.464272   9.464368   9.464463 ]
 [ 2.3709533  4.767527   4.970966  ...  9.779315   9.779721   9.781188 ]
 [ 2.4757743  4.67912    4.856323  ...  9.271653   9.271903   9.272864 ]
 ...
 [ 4.972886   5.0007386  5.1257625 ...  8.074588   8.075148   8.075668 ]
 [ 4.0512795  4.0792313  4.126018  ...  7.968236   7.970854   7.9710336]
 [10.425038  10.818621  11.25877   ... 15.417514  15.417756  15.418318 ]]
time cost: 177.1195511817932


## Search for the full features

In [9]:
feature = features[:407460]

In [10]:
start = time.time()
k = 2000                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[1335466853 1344065098 1381864259 ... 1381724146  937015705 1324020770]
 [1335466854 1338353227 1361070531 ... 1338602645 1340925617 1365704499]
 [1335466855 1335466856 1335466857 ... 1339076546 1324020753 1344029892]
 ...
 [1338070435 1333011994 1334117224 ... 1346125291 1358547569 1383306463]
 [1337711432 1343386772 1376442074 ... 1340404078 1370866233 1362234164]
 [1376442162 1380317422 1326831842 ... 1319703164 1382776830 4373929790]]
[[1.3314159 4.8101816 4.849719  ... 9.464272  9.464368  9.464463 ]
 [2.3709533 4.767527  4.970966  ... 9.779315  9.779721  9.781188 ]
 [2.4757743 4.67912   4.856323  ... 9.271653  9.271903  9.272864 ]
 ...
 [2.3624344 3.05092   3.1213388 ... 5.7416854 5.7419963 5.7423205]
 [2.9153843 3.2243872 3.4649186 ... 6.250318  6.250673  6.2506847]
 [3.9846787 4.777871  5.2578726 ... 9.382974  9.383143  9.38435  ]]
time cost: 1696.3735303878784


In [11]:
import joblib

In [13]:
x = joblib.load('order.pkl')

In [33]:
offset_dict = {}
offset = 0

for data in x:
    offset_dict[data[0][:-26]] = offset
    print(data[0][:-26])
    with h5py.File(data[0], "r") as f:
        offset += len(f['chunks'])

Books3_ftfy_cleaned_id_shuf
rn_dedup_shuf_cleaned_0.7_cleaned_shuf
OpenWebText2_ftfy_cleaned_id_shuf
StackExchange_ftfy_id_shuf
stories_dedup0.7_shuf_cleaned_shuf
PubMed_Abstracts_ftfy_id_shuf
Wikipedia_en_ftfy_id_shuf
Gutenberg_PG-19_ftfy_cleaned_id_cleaned_shuf
BookCorpus2_ftfy_cleaned_id_shuf
NIH_ExPorter_ftfy_id_shuf
CC-2020-50_id_cleaned_shuf
Pile-CC_id_cleaned_shuf
CC-2021-04_id_cleaned_shuf
ArXiv_ftfy_cleaned_id_shuf
Github_ftfy_id_shuf


In [35]:
offset_dict

{'Books3_ftfy_cleaned_id_shuf': 0,
 'rn_dedup_shuf_cleaned_0.7_cleaned_shuf': 393262503,
 'OpenWebText2_ftfy_cleaned_id_shuf': 742721270,
 'StackExchange_ftfy_id_shuf': 976632233,
 'stories_dedup0.7_shuf_cleaned_shuf': 1161539157,
 'PubMed_Abstracts_ftfy_id_shuf': 1242369844,
 'Wikipedia_en_ftfy_id_shuf': 1317366217,
 'Gutenberg_PG-19_ftfy_cleaned_id_cleaned_shuf': 1383997021,
 'BookCorpus2_ftfy_cleaned_id_shuf': 1424811327,
 'NIH_ExPorter_ftfy_id_shuf': 1448390166,
 'CC-2020-50_id_cleaned_shuf': 1453625626,
 'Pile-CC_id_cleaned_shuf': 2542325217,
 'CC-2021-04_id_cleaned_shuf': 3328832748,
 'ArXiv_ftfy_cleaned_id_shuf': 4638515069,
 'Github_ftfy_id_shuf': 4957650202}

In [34]:
joblib.dump(offset_dict, 'offset_dict.pkl')

['offset_dict.pkl']

In [11]:
start = time.time()
k = 20                                # we want to see 4 nearest neighbors
D, I = index.search(feature, k)     # sanity check
print(I)
print(D)
end = time.time()

print("time cost:", end - start)

[[62266347 20303550  1745174 ... 40950489 62339080 31828088]
 [20303550 39125951 40950489 ... 62339080 38588580 15447837]
 [20303550  1745174 39125951 ...  9334423 18528054 16748728]
 ...
 [31828088 27093283 22120375 ... 57218540 44183648 37273546]
 [31828088 27093283  4505045 ... 44183648 37273546 28488904]
 [13788211 11217483 37273546 ... 20303550  3994002 31828088]]
[[207.53688 208.81802 211.51538 ... 213.64128 214.6185  217.52325]
 [184.92316 193.34473 194.96954 ... 197.93991 198.66647 199.72974]
 [178.20432 180.7522  182.91786 ... 187.12524 187.12524 187.12524]
 ...
 [182.35732 182.36682 183.87283 ... 185.88074 185.88074 185.88074]
 [180.05756 182.2111  182.55023 ... 189.24548 189.24548 189.24548]
 [167.90437 167.90437 167.90437 ... 168.34467 168.8721  170.76395]]
time cost: 623.9999520778656


In [14]:
I.shape

(40746, 20)

In [16]:
start = time.time()
f = h5py.File('/gpfs/fs1/projects/gpu_adlr/datasets/boxinw/pretrained_data/Wikipedia-shuf/Wikipedia_en_ftfy_id_shuf_text_document.tokens.h5py', "r")
tokens = f['tokens']
# f.close()
end = time.time()

In [17]:
tokens.shape

(2037312, 2048)

In [18]:
tokens[0][:64]

array([27594,   870,   309,    13,    40,  1539,  6960,   449,  1453,
        7357,  1222,   509,   589,   352,     8,   784,   513,    25,
        3559,   198,     1,  5756,   338,  1514,     1,   357, 27594,
         870,  1815, 12523,  1222, 16342,  5966,     8,   357, 11547,
         771,   416,  4403,    70,   360,  1222,  5395,   449,   684,
         259,     8,   784,   513,    25,  3510,   198,     1,    38,
         648, 38031, 32020,   259, 29653,   784,   513,    25,  2231,
         198], dtype=uint16)

In [19]:
import sys
sys.path.append("/home/boxinw-src/megatron-lm/megatron")
sys.path.append("/home/boxinw-src/megatron-lm/")

In [36]:
from megatron.tokenizer import build_tokenizer
from megatron.data import indexed_dataset

from megatron.data.indexed_dataset import make_dataset as make_indexed_dataset


In [37]:
import argparse

def get_args():
    parser = argparse.ArgumentParser()
    group = parser.add_argument_group(title='input data')
    group.add_argument('--input', type=str, required=False,
                       help='Path to input JSON')
    group.add_argument('--json-keys', nargs='+', default=['text'],
                       help='space separate listed of keys to extract from json')
    group.add_argument('--split-sentences', action='store_true',
                       help='Split documents into sentences.')
    group.add_argument('--keep-newlines', action='store_true',
                       help='Keep newlines between sentences when splitting.')

    group = parser.add_argument_group(title='tokenizer')
    group.add_argument('--tokenizer-type', type=str, required=False,
                       choices=['BertWordPieceLowerCase','BertWordPieceCase',
                                'GPT2BPETokenizer'],
                       help='What type of tokenizer to use.')
    group.add_argument('--vocab-file', type=str, default=None,
                       help='Path to the vocab file')
    group.add_argument('--merge-file', type=str, default=None,
                       help='Path to the BPE merge file (if necessary).')
    group.add_argument('--append-eod', action='store_true',
                       help='Append an <eod> token to the end of a document.')


    group = parser.add_argument_group(title='output data')
    group.add_argument('--output-prefix', type=str, required=False,
                       help='Path to binary output file without suffix')
    group.add_argument('--dataset-impl', type=str, default='mmap',
                       choices=['lazy', 'cached', 'mmap'])

    group = parser.add_argument_group(title='runtime')
    group.add_argument('--workers', type=int, default=1,
                       help='Number of worker processes to launch')
    group.add_argument('--log-interval', type=int, default=100,
                       help='Interval between progress updates')
    group.add_argument('-f', type=str, default='',
                   help='Make jupyter happy')
    args = parser.parse_args()
    args.keep_empty = False

#     if args.tokenizer_type.lower().startswith('bert'):
#         if not args.split_sentences:
#             print("Bert tokenizer detected, are you sure you don't want to split sentences?")

    # some default/dummy values for the tokenizer
    args.rank = 0
    args.make_vocab_size_divisible_by = 128
    args.tensor_model_parallel_size = 1
    args.vocab_extra_ids = 0

    return args

args = get_args()

In [38]:
args.tokenizer_type = "GPT2BPETokenizer"
args.vocab_file = "/home/boxinw-src/megatron-lm//gpt2-vocab.json"
args.merge_file = "/home/boxinw-src/megatron-lm/gpt2-merges.txt"

tokenizer = build_tokenizer(args)

> building GPT2BPETokenizer tokenizer ...
 > padded vocab (size: 50257) with 47 dummy tokens (new size: 50304)


In [40]:
start = time.time()
f = h5py.File('/gpfs/fs1/projects/gpu_adlr/datasets/boxinw/pretrained_data/Wikipedia-shuf/Wikipedia_en_ftfy_id_shuf_text_document.tokens.h5py', "r")
tokens = f['tokens']
# f.close()
end = time.time()

In [41]:
sample3 = tokenizer.detokenize(tokens[0][64:128])

In [42]:
sample3

'"These Are the Daze" – 3:44\n"I Wanna Sang" – 4:36\n"The Children\'s Song" (Produced by Gorilla Tek) – 3:58\n"U Neva Know" – 4:19\n"Sugar (Gimme Some)" (feat'

In [43]:
I[1]

array([1335466854, 1338353227, 1361070531, ..., 1338602645, 1340925617,
       1365704499])

In [45]:
start = time.time()
f2 = h5py.File('/gpfs/fs1/projects/gpu_adlr/datasets/boxinw/processed_data/chunks/pretraining_corpus.chunks.hdf5', "r")
database = f2['chunks']
# f.close()
end = time.time()

In [46]:
neighbor1 = tokenizer.detokenize(database[1335466854])

In [47]:
neighbor1

'"Gangsta Livin\'" – 3:45\n"These Are the Daze" – 3:44\n"I Wanna Sang" – 4:36\n"The Children\'s Song" (Produced by Gorilla Tek) – 3:58\n"U Neva Know" – 4:19'

In [48]:
neighbor2 = tokenizer.detokenize(database[1338353227])

In [49]:
neighbor2

'" – 2:01\n"Azzville" – 3:13\n"September Love" – 4:06\n"Bad Girl" – 2:58\n"Sergeant Louise" – 3:54\n"U Make Me Say" (Heya Remix) (Featuring Ja Rule) – 3:'

In [51]:
neighbor3 = tokenizer.detokenize(database[1361070531])

In [52]:
neighbor3

'" – 3:53\n"Put It on You" – 4:04\n"Propa" – 4:12 (featuring Beenie Man)\n"It\'s All Yours" – 4:41 (featuring Gina Thompson)\n"I Can\'t Make a Mistake" – 3:51'